In [1]:
import arviz as az
import cmdstanpy
import numpy as np
import xarray as xr

from dms_stan.datasets.trpb import load_trpb_dataset, TrpBGrowthModel
from dms_stan.model.stan.stan_results import SampleResults
from dms_stan.model.components import Normal

In [2]:
test = SampleResults.from_disk('/home/bwittmann/GitRepos/DMSStan/flip3/trpB/tmp/model-20250221204218_arviz.nc')
diagnostics = test.diagnose()

In [ ]:
test.evaluate_sample_stats()

<xarray.Dataset> Size: 204kB
Dimensions:          (chain: 4, draw: 1000)
Coordinates:
  * chain            (chain) int64 32B 0 1 2 3
  * draw             (draw) int64 8kB 0 1 2 3 4 5 6 ... 994 995 996 997 998 999
Data variables:
    lp               (chain, draw) float64 32kB ...
    acceptance_rate  (chain, draw) float64 32kB ...
    step_size        (chain, draw) float64 32kB ...
    tree_depth       (chain, draw) int64 32kB 8 8 8 8 8 8 8 8 ... 8 8 8 8 8 8 8
    n_steps          (chain, draw) int64 32kB ...
    diverging        (chain, draw) bool 4kB ...
    energy           (chain, draw) float64 32kB 4.153e+08 ... 4.153e+08
Attributes:
    created_at:                 2025-03-10T16:52:10.649748+00:00
    arviz_version:              0.20.0
    inference_library:          cmdstanpy
    inference_library_version:  1.2.5

In [ ]:
test.evaluate_diagnostic_stats()

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> observed_data
	> constant_data
	> diagnostics